In [1]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-05 13:50:59,932 INFO: Initializing external client
2025-03-05 13:50:59,934 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 13:51:00,992 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683
Fetching data from 2025-02-04 18:50:59.925208+00:00 to 2025-03-05 17:50:59.925208+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (57.19s) 


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 13:52:36,988 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 13:52:37,014 INFO: Initializing external client
2025-03-05 13:52:37,016 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 13:52:37,795 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [4]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1538, in _execute_child
   

In [5]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,263,52.0,2025-03-05 19:00:00+00:00
1,255,2.0,2025-03-05 19:00:00+00:00
2,210,0.0,2025-03-05 19:00:00+00:00
3,147,0.0,2025-03-05 19:00:00+00:00
4,217,1.0,2025-03-05 19:00:00+00:00
...,...,...,...
247,73,0.0,2025-03-05 19:00:00+00:00
248,12,1.0,2025-03-05 19:00:00+00:00
249,45,10.0,2025-03-05 19:00:00+00:00
250,53,0.0,2025-03-05 19:00:00+00:00


In [6]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-05 13:52:56,928 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 13:52:56,945 INFO: Initializing external client
2025-03-05 13:52:56,948 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 13:52:57,730 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [7]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1215683/fs/1203307/fg/1403567


Uploading Dataframe: 100.00% |██████████| Rows 252/252 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215683/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)